In [1]:
import time
import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn import preprocessing
from sklearn.neural_network import MLPRegressor
from sklearn import model_selection

Config parameters:

In [2]:
# Path to datasets
train_dataset_name = './data/processed/credit_train.csv'
test_dataset_name = './data/processed/credit_test.csv'

# Target directory for saving
target_dir = 'PSDA/data/predictions/'
name_of_predictions_file = 'credit_test.csv'

# Run hyperparameter optimization (with a given random state it is unnecessary to compute each time)
hyper_opt = True

## Load Data and make temporary train-test-split

In [3]:
df_train = pd.read_csv(train_dataset_name)
df_test = pd.read_csv(test_dataset_name)

print(df_train.columns)
print(df_test.columns)

Index(['Unnamed: 0', 'Income', 'Limit', 'Rating', 'Cards', 'Balance',
       'Student_Yes'],
      dtype='object')
Index(['Unnamed: 0', 'Income', 'Limit', 'Rating', 'Cards', 'Student_Yes'], dtype='object')


In [4]:
train = df_train.values.astype(float)
train_ids = train[:, 0]
train_balance = train[:, list(df_train.columns).index('Balance')]
train_data = train[:, [c != 'Balance' and c != 'Unnamed: 0' for c in df_train.columns]]

test = df_test.values.astype(float)
test_ids = test[:, 0]
test_data = test[:, [c != 'Unnamed: 0' for c in df_test.columns]]

print(train.shape)
print(train_ids.shape)
print(train_balance.shape)
print(train_data.shape)
print(test.shape)
print(test_ids.shape)
print(test_data.shape)

(350, 7)
(350,)
(350,)
(350, 5)
(50, 6)
(50,)
(50, 5)


In [5]:
np.mean(train_balance)

521.7085714285714

In [6]:
# temp_train_ids, temp_test_ids, temp_train_data, temp_test_data, temp_train_balance, temp_test_balance = model_selection.train_test_split(train_ids, train_data, train_balance, test_size=50, random_state=4)

# Shuffle data to use K-Fold cross-validation in the following 
indicis = np.arange(train_data.shape[0])
np.random.seed(6)
np.random.shuffle(indicis)
random_train_balance = train_balance[indicis]
random_train_data = train_data[indicis]

Findings: By only looking at a single validation dataset, the result on the test data is not as meaningful, which is why I use K-Fold cross-validation below. 

## Preprocess data

In [7]:
# This code was from the static validation set and therefore is commented out. 
# snv = preprocessing.StandardScaler()
# prep_temp_train_data = np.c_[snv.fit_transform(temp_train_data[:, :6]), temp_train_data[:, 6:]]
# prep_temp_test_data = np.c_[snv.transform(temp_test_data[:, :6]), temp_test_data[:, 6:]]

# Only apply scaler on numerical columns
snv = preprocessing.StandardScaler()
prep_train_data = np.c_[snv.fit_transform(random_train_data[:, :6]), random_train_data[:, 6:]]
prep_test_data = np.c_[snv.fit_transform(test_data[:, :6]), test_data[:, 6:]]

## Create MLP Regressor instances 

Note: As done so in for the Gradient Boosting algorithm, I set every negative prediction to zero.

In [10]:
class MLPWithCutoff():
    
    def __init__(self, learning_rate_init, batch_size, alpha, beta_1, beta_2, hidden_layer_sizes=None, activation='relu', *, solver='adam', learning_rate='constant', power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, epsilon=1e-08, n_iter_no_change=10, max_fun=15000):
        self.mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, activation='relu', solver='adam', alpha=alpha, batch_size=int(batch_size), learning_rate='constant', learning_rate_init=learning_rate_init, power_t=0.5, max_iter=5000, shuffle=True, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=beta_1, beta_2=beta_2, epsilon=1e-08, n_iter_no_change=10, max_fun=15000, random_state=7)
        
    def fit(self, X, y):
        self.mlp_regressor.fit(X, y)
    
    def predict(self, X):
        prediction = self.mlp_regressor.predict(X)
        prediction = prediction * np.array(prediction > 0, dtype=int)
        return prediction
    
    def get_params(self, deep=True):
        return self.mlp_regressor.get_params(deep=deep)

Note: In the following Bayesian optimization is used to find the optimal hyperparameter. The parameter bounds are:
* 'learning_rate_init': (0.000001, 0.1)
    - I want to explore the neighbourhood of the default value that is why I choose 0.1 as an upper bound and 0.000001 as an lower bound.
* 'batch_size': (1, 100) 
    - The batch_size has as upper bound 100 because in the train dataset are only 350 samples.
* 'alpha': (0.000001, 0.1)
    - I want to explore the neighbourhood of the default value that is why I choose 0.1 as an upper bound and 0.000001 as an lower bound.
* 'beta_1': (0.8, 0.999)
    - I want to explore the neighbourhood of the default value that is why I choose 0.8 as an upper bound and 0.999 as an lower bound.
* 'beta_2': (0.9, 0.999999)
    - I want to explore the neighbourhood of the default value that is why I choose 0.9 as an upper bound and 0.999999 as an lower bound.
* 'hidden_layer_count': (0, 20)
    - The architecture is [16] + [20] * hidden_layer_count + [16]. I want to set the upper bound of layers as high es possible.

In [8]:
def hyperparameter_function_mlp_with_cut(learning_rate_init, batch_size, alpha, beta_1, beta_2, hidden_layer_count):
    """ Function for hyperparameter optimization
    """    
    hidden_layer_sizes = [16]
    hidden_layer_sizes.extend([64] * int(hidden_layer_count))
    hidden_layer_sizes.append(16)      
    # Learning rate is fixed
    mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, batch_size=int(batch_size), learning_rate_init=learning_rate_init, beta_1=beta_1, beta_2=beta_2, max_iter=15000, random_state=7)
    scores = model_selection.cross_val_score(mlp_regressor, X=prep_train_data, y=random_train_balance, cv=5, scoring='neg_mean_absolute_error')
    return np.min(scores)


# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.000001, 0.1), 'batch_size': (1, 100), 'alpha': (0.000001, 0.1), 'beta_1': (0.8, 0.999), 'beta_2': (0.9, 0.999999), 'hidden_layer_count': (0, 20)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_mlp_with_cut,
    pbounds=pbounds,
    random_state=7,
)
if hyper_opt:
    optimizer.maximize(
        init_points=50,
        n_iter=100,
    )

|   iter    |  target   |   alpha   | batch_... |  beta_1   |  beta_2   | hidden... | learni... |
-------------------------------------------------------------------------------------------------
| 1         | -631.8    | 0.007632  | 78.21     | 0.8872    | 0.9723    | 19.56     | 0.05385   |
| 2         | -418.0    | 0.05011   | 8.133     | 0.8534    | 0.95      | 13.58     | 0.08037   |
| 3         | -97.99    | 0.03809   | 7.528     | 0.8573    | 0.991     | 4.268     | 0.04521   |
| 4         | -165.1    | 0.09312   | 3.465     | 0.9195    | 0.995     | 4.606     | 0.05485   |
| 5         | -396.3    | 0.09091   | 14.18     | 0.9042    | 0.975     | 13.38     | 0.04678   |
| 6         | -83.38    | 0.02049   | 49.59     | 0.8741    | 0.9477    | 7.318     | 0.08379   |
| 7         | -45.68    | 0.07686   | 32.09     | 0.914     | 0.9276    | 9.057     | 0.0353    |
| 8         | -397.6    | 0.06574   | 37.66     | 0.8914    | 0.9719    | 8.26      | 0.09064   |
| 9         | -395.5

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 65        | -242.3    | 0.03206   | 53.32     | 0.9776    | 0.9362    | 7.082     | 1e-06     |
| 66        | -71.27    | 0.08572   | 49.32     | 0.8586    | 0.9605    | 19.35     | 0.00656   |
| 67        | -533.1    | 0.009911  | 32.96     | 0.8581    | 0.9746    | 15.09     | 0.0948    |
| 68        | -413.3    | 0.02831   | 36.53     | 0.8892    | 0.9299    | 13.94     | 0.05016   |
| 69        | -609.8    | 0.02193   | 40.28     | 0.8826    | 0.9647    | 14.38     | 0.02156   |
| 70        | -41.25    | 0.05269   | 38.98     | 0.96      | 0.9382    | 11.26     | 0.008871  |
| 71        | -633.2    | 0.09896   | 40.3      | 0.961     | 0.9512    | 18.91     | 0.06421   |
| 72        | -161.3    | 0.02794   | 56.76     | 0.8719    | 0.9297    | 8.65      | 0.06015   |
| 73        | -27.86    | 0.05083   | 56.34     | 0.8517    | 0.9492    | 4.4       | 0.01901   |
| 74        | -53.52    | 0.02835   | 59.92     | 0.889     | 0.9491    | 5.78      | 0.0329    |
| 75        | -63.5 

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 84        | -48.04    | 0.02635   | 63.11     | 0.8       | 1.0       | 16.8      | 1e-06     |
| 85        | -142.3    | 0.09212   | 31.88     | 0.926     | 0.9516    | 18.48     | 0.01591   |
| 86        | -32.6     | 0.02481   | 68.2      | 0.8189    | 0.9333    | 17.13     | 8.773e-05 |
| 87        | -548.7    | 0.003283  | 71.69     | 0.8396    | 0.9547    | 18.23     | 0.0765    |
| 88        | -25.72    | 0.01229   | 77.19     | 0.8857    | 0.9753    | 0.9237    | 0.06925   |
| 89        | -164.7    | 0.03362   | 74.92     | 0.9868    | 0.9244    | 3.811     | 0.01438   |
| 90        | -2.717e+0 | 0.04604   | 74.55     | 0.9961    | 0.9506    | 7.406     | 0.06787   |
| 91        | -62.19    | 0.03637   | 78.06     | 0.8968    | 0.9391    | 6.416     | 0.05932   |
| 92        | -409.7    | 0.06945   | 77.13     | 0.949     | 0.9763    | 10.2      | 0.05816   |
| 93        | -57.94    | 0.03      | 80.41     | 0.8941    | 0.9043    | 12.18     | 0.004821  |
| 94        | -519.6

...

| 107       | -236.0    | 0.1       | 86.88     | 0.8       | 0.985     | 13.25     | 1e-06     |
| 108       | -111.5    | 0.02395   | 87.79     | 0.9751    | 0.9046    | 0.0627    | 0.09121   |


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 109       | -251.0    | 0.1       | 90.92     | 0.8       | 0.9       | 16.94     | 1e-06     |
| 110       | -398.9    | 0.05216   | 87.38     | 0.8767    | 0.9094    | 16.92     | 0.05183   |
| 111       | -116.4    | 0.06707   | 1.079     | 0.8896    | 0.9588    | 7.214     | 0.03802   |
| 112       | -85.44    | 0.007429  | 64.12     | 0.8647    | 0.9559    | 2.455     | 0.08526   |
| 113       | -79.73    | 0.07719   | 60.52     | 0.8693    | 0.9711    | 2.322     | 0.08189   |
| 114       | -16.19    | 0.02465   | 56.87     | 0.9506    | 0.9806    | 0.8155    | 0.09076   |
| 115       | -31.4     | 0.08061   | 68.23     | 0.9046    | 0.976     | 0.4313    | 0.03938   |
| 116       | -629.8    | 0.01621   | 60.57     | 0.8966    | 0.9466    | 19.38     | 0.07797   |
| 117       | -632.2    | 0.02944   | 65.61     | 0.9729    | 0.9936    | 19.6      | 0.0596    |
| 118       | -160.0    | 0.05753   | 26.25     | 0.9042    | 0.9513    | 14.95     | 0.02185   |
| 119       | -480.1

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 145       | -498.4    | 0.1       | 97.45     | 0.999     | 0.9       | 11.6      | 1e-06     |
| 146       | -632.0    | 1e-06     | 97.62     | 0.8       | 1.0       | 15.11     | 0.1       |
| 147       | -23.43    | 0.01443   | 99.86     | 0.9103    | 0.9927    | 6.821     | 0.0103    |
| 148       | -17.03    | 0.0194    | 18.61     | 0.8205    | 0.9163    | 0.0563    | 0.01481   |
| 149       | -630.9    | 1e-06     | 97.65     | 0.8       | 0.9997    | 18.76     | 0.1       |
| 150       | -81.39    | 0.08784   | 92.96     | 0.8451    | 0.9515    | 19.76     | 0.004665  |


Note: I adjust the 'learning_rate_init' and 'batch_size' interval to search in the area where the previous best model was found. 

In [9]:
def hyperparameter_function_mlp_with_cut(learning_rate_init, batch_size, alpha, beta_1, beta_2):
    """ Function for hyperparameter optimization
    """    
    hidden_layer_sizes = [16]
    hidden_layer_sizes.extend([64] * int(7))
    hidden_layer_sizes.append(16)      
    # Learning rate is fixed
    mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, batch_size=int(batch_size), learning_rate_init=learning_rate_init, beta_1=beta_1, beta_2=beta_2, max_iter=5000, random_state=4)
    scores = model_selection.cross_val_score(mlp_regressor, X=prep_train_data, y=random_train_balance, cv=5, scoring='neg_mean_absolute_error')
    return np.min(scores)
    

# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.0001, 0.1), 'batch_size': (60, 100), 'alpha': (0.000001, 0.1), 'beta_1': (0.8, 0.999), 'beta_2': (0.9, 0.999999)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_mlp_with_cut,
    pbounds=pbounds,
    random_state=14,
)
if hyper_opt:
    optimizer.maximize(
        init_points=100,
        n_iter=200,
    )

|   iter    |  target   |   alpha   | batch_... |  beta_1   |  beta_2   | learni... |
-------------------------------------------------------------------------------------
| 1         | -137.9    | 0.05139   | 90.93     | 0.9732    | 0.9008    | 0.03104   |
| 2         | -96.59    | 0.09576   | 80.52     | 0.8633    | 0.9539    | 0.0222    |
| 3         | -100.1    | 0.08065   | 73.69     | 0.9072    | 0.9006    | 0.06735   |
| 4         | -95.78    | 0.021     | 97.3      | 0.8745    | 0.9752    | 0.07634   |
| 5         | -75.61    | 0.08705   | 64.4      | 0.8597    | 0.9475    | 0.06733   |
| 6         | -87.65    | 0.02576   | 88.05     | 0.9307    | 0.9182    | 0.03227   |
| 7         | -94.52    | 0.08875   | 63.33     | 0.9486    | 0.9062    | 0.07317   |
| 8         | -233.6    | 0.07425   | 76.32     | 0.9973    | 0.9452    | 0.01907   |
| 9         | -105.5    | 0.03516   | 88.59     | 0.8711    | 0.9052    | 0.03166   |
| 10        | -106.1    | 0.09988   | 71.52     | 0.89

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5000) reached and the optimization hasn't converged yet.
  warnings.warn(


| 179       | -389.1    | 0.1       | 73.97     | 0.8       | 1.0       | 0.1       |
| 180       | -49.76    | 0.07897   | 94.01     | 0.9269    | 0.9895    | 0.04632   |
| 181       | -37.43    | 0.009995  | 96.2      | 0.9825    | 0.9977    | 0.0266    |
| 182       | -5.166e+0 | 1e-06     | 92.4      | 0.999     | 0.9       | 0.0001    |
| 183       | -37.89    | 0.08153   | 91.88     | 0.8101    | 0.9955    | 0.02895   |
| 184       | -3.754e+0 | 1e-06     | 90.29     | 0.999     | 0.9       | 0.0001    |
| 185       | -383.8    | 0.1       | 86.23     | 0.999     | 1.0       | 0.0001    |
| 186       | -27.59    | 1e-06     | 61.23     | 0.8       | 1.0       | 0.0001    |
| 187       | -106.5    | 0.07024   | 73.68     | 0.9138    | 0.9082    | 0.07248   |
| 188       | -674.4    | 1e-06     | 60.82     | 0.999     | 0.9       | 0.1       |
| 189       | -131.9    | 0.04635   | 96.63     | 0.8225    | 0.9748    | 0.04337   |
| 190       | -25.91    | 0.0736    | 99.07     | 0.82

Note: Now I changed the architecture the optimization is not only done on the layer count but also on the layer size. To restrict the search area, all layers have the same size.

In [ ]:
def hyperparameter_function_mlp_with_cut(learning_rate_init, batch_size, alpha, beta_1, beta_2, layer_count, hidden_size):
    """ Function for hyperparameter optimization
    """    
    hidden_layer_sizes = [int(hidden_size)] * int(layer_count)
    # Learning rate is fixed
    mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, batch_size=int(batch_size), learning_rate_init=learning_rate_init, beta_1=beta_1, beta_2=beta_2, max_iter=15000, random_state=7)
    scores = model_selection.cross_val_score(mlp_regressor, X=prep_train_data, y=random_train_balance, cv=5, scoring='neg_mean_absolute_error')
    return np.min(scores)


# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.000001, 0.1), 'batch_size': (1, 300), 'alpha': (0.000001, 0.1), 'beta_1': (0.8, 0.999), 'beta_2': (0.9, 0.999999), 'layer_count': (1, 5), 'hidden_size': (64, 128)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_mlp_with_cut,
    pbounds=pbounds,
    random_state=7,
)
if hyper_opt:
    optimizer.maximize(
        init_points=50,
        n_iter=100,
    )

|   iter    |  target   |   alpha   | batch_... |  beta_1   |  beta_2   | hidden... | layer_... | learni... |
-------------------------------------------------------------------------------------------------------------
| 1         | -62.78    | 0.007632  | 234.2     | 0.8872    | 0.9723    | 126.6     | 3.154     | 0.05011   |
| 2         | -18.75    | 0.007206  | 81.26     | 0.8995    | 0.9679    | 115.4     | 2.524     | 0.006595  |
| 3         | -28.16    | 0.02882   | 273.0     | 0.8425    | 0.9452    | 123.6     | 1.1       | 0.06006   |
| 4         | -45.02    | 0.09501   | 69.86     | 0.9091    | 0.9909    | 72.52     | 3.094     | 0.07504   |
| 5         | -29.25    | 0.0669    | 140.9     | 0.8408    | 0.9491    | 87.83     | 2.91      | 0.03659   |
| 6         | -26.72    | 0.08379   | 230.8     | 0.8625    | 0.9573    | 81.67     | 2.811     | 0.0353    |
| 7         | -34.14    | 0.06574   | 111.7     | 0.8914    | 0.9719    | 90.43     | 4.626     | 0.01805   |
| 8       

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is go

| 31        | -630.5    | 0.09735   | 286.7     | 0.8845    | 0.9594    | 66.54     | 4.955     | 0.08187   |
| 32        | -21.81    | 0.06365   | 228.6     | 0.8374    | 0.9308    | 79.77     | 3.384     | 0.009191  |
| 33        | -620.8    | 0.08956   | 139.2     | 0.8885    | 0.9105    | 107.8     | 4.268     | 0.06296   |
| 34        | -29.15    | 0.0242    | 235.8     | 0.829     | 0.9827    | 101.2     | 2.157     | 0.05132   |
| 35        | -22.4     | 0.06289   | 78.32     | 0.9685    | 0.9421    | 121.1     | 4.342     | 0.009936  |
| 36        | -49.72    | 0.06463   | 93.84     | 0.9501    | 0.9543    | 93.29     | 4.581     | 0.005726  |
| 37        | -534.6    | 0.05573   | 99.05     | 0.807     | 0.9754    | 99.95     | 4.577     | 0.05983   |


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is go

| 38        | -17.26    | 0.0337    | 295.6     | 0.823     | 0.9053    | 110.9     | 2.483     | 0.03615   |
| 39        | -50.98    | 0.08765   | 98.86     | 0.9769    | 0.9644    | 85.06     | 1.238     | 0.02451   |
| 40        | -19.81    | 0.09684   | 122.2     | 0.8318    | 0.9298    | 121.6     | 1.66      | 0.07779   |


...

| 80        | -631.4    | 1e-06     | 300.0     | 0.8       | 0.9       | 64.0      | 1.0       | 1e-06     |
| 81        | -18.21    | 0.02868   | 226.7     | 0.8801    | 0.9076    | 114.1     | 1.512     | 0.08498   |
| 82        | -295.2    | 0.006638  | 271.9     | 0.9794    | 0.9139    | 64.84     | 2.876     | 0.01044   |
| 83        | -30.32    | 0.08306   | 102.2     | 0.919     | 0.9007    | 64.24     | 4.862     | 0.09488   |
| 84        | -21.38    | 0.08283   | 195.3     | 0.8782    | 0.9457    | 64.07     | 2.159     | 0.07091   |
| 85        | -24.92    | 1e-06     | 257.6     | 0.8       | 1.0       | 82.16     | 1.0       | 0.1       |
| 86        | -29.65    | 0.02942   | 12.3      | 0.9338    | 0.9753    | 110.9     | 2.656     | 0.0609    |


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:609: UserWarning: Got `batch_size` 

| 87        | -630.4    | 0.1       | 300.0     | 0.999     | 1.0       | 128.0     | 5.0       | 1e-06     |
| 88        | -49.22    | 0.04572   | 59.97     | 0.836     | 0.9582    | 81.17     | 4.25      | 0.08121   |
| 89        | -15.99    | 0.006159  | 81.82     | 0.9488    | 0.9362    | 100.7     | 1.873     | 0.0174    |
| 90        | -16.27    | 0.0979    | 169.0     | 0.8163    | 0.9851    | 114.3     | 1.029     | 0.02625   |


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 91        | -631.2    | 1e-06     | 123.1     | 0.999     | 0.9       | 83.34     | 1.0       | 1e-06     |
| 92        | -15.96    | 0.02385   | 190.8     | 0.8309    | 0.964     | 83.35     | 1.252     | 0.002418  |
| 93        | -33.61    | 0.05491   | 20.18     | 0.9939    | 0.9989    | 64.01     | 4.62      | 0.06765   |
| 94        | -115.0    | 1e-06     | 26.94     | 0.999     | 1.0       | 113.9     | 1.0       | 0.1       |
| 95        | -122.2    | 0.02273   | 153.6     | 0.971     | 0.9325    | 90.83     | 2.553     | 0.08017   |
| 96        | -30.34    | 0.04036   | 127.3     | 0.8205    | 0.9342    | 64.3      | 3.36      | 0.02457   |
| 97        | -24.44    | 0.01461   | 245.0     | 0.805     | 0.9107    | 64.19     | 2.233     | 0.03461   |


Note: After waiting for a couple of hours I interrupted the run above. Hereafter, I tried to optimize the best architecture. But I still allow the layer size to increase.

In [8]:
def hyperparameter_function_mlp_with_cut(learning_rate_init, batch_size, alpha, beta_1, beta_2, hidden_size):
    """ Function for hyperparameter optimization
    """    
    hidden_layer_sizes = [int(hidden_size)] * 1
    # Learning rate is fixed
    mlp_regressor = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, batch_size=int(batch_size), learning_rate_init=learning_rate_init, beta_1=beta_1, beta_2=beta_2, max_iter=15000, random_state=7)
    scores = model_selection.cross_val_score(mlp_regressor, X=prep_train_data, y=random_train_balance, cv=5, scoring='neg_mean_absolute_error')
    return np.min(scores)


# Bounded region of parameter space
pbounds = {'learning_rate_init': (0.0001, 0.01), 'batch_size': (150, 250), 'alpha': (0.000001, 0.1), 'beta_1': (0.8, 0.999), 'beta_2': (0.9, 0.999999), 'hidden_size': (115, 130)}

optimizer = BayesianOptimization(
    f=hyperparameter_function_mlp_with_cut,
    pbounds=pbounds,
    random_state=25,
)
if hyper_opt:
    optimizer.maximize(
        init_points=50,
        n_iter=100,
    )

|   iter    |  target   |   alpha   | batch_... |  beta_1   |  beta_2   | hidden... | learni... |
-------------------------------------------------------------------------------------------------
| 1         | -16.87    | 0.08701   | 208.2     | 0.8555    | 0.9186    | 121.2     | 0.001262  |
| 2         | -15.73    | 0.0685    | 193.8     | 0.9107    | 0.9367    | 121.0     | 0.001219  |
| 3         | -16.3     | 0.0447    | 208.5     | 0.8322    | 0.9521    | 119.9     | 0.007022  |
| 4         | -19.04    | 0.03664   | 233.6     | 0.8958    | 0.9516    | 120.7     | 0.009976  |
| 5         | -16.2     | 0.05142   | 205.9     | 0.8069    | 0.972     | 121.3     | 0.004426  |
| 6         | -20.91    | 0.02817   | 240.0     | 0.9333    | 0.9456    | 119.3     | 0.005306  |
| 7         | -17.27    | 0.05592   | 224.5     | 0.9648    | 0.9824    | 116.2     | 0.006484  |
| 8         | -45.13    | 0.03093   | 202.4     | 0.9907    | 0.9883    | 119.4     | 0.005173  |
| 9         | -16.59

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 21        | -16.18    | 0.04096   | 161.0     | 0.8171    | 0.9748    | 117.7     | 0.0002494 |
| 22        | -21.96    | 0.09138   | 242.6     | 0.916     | 0.9313    | 130.0     | 0.001585  |
| 23        | -15.55    | 0.09945   | 162.5     | 0.8699    | 0.9531    | 119.9     | 0.008084  |
| 24        | -16.24    | 0.09603   | 203.9     | 0.8994    | 0.9801    | 127.0     | 0.001287  |
| 25        | -17.04    | 0.0246    | 212.7     | 0.8751    | 0.9063    | 120.6     | 0.0008337 |
| 26        | -16.33    | 0.004445  | 179.3     | 0.9502    | 0.9953    | 123.8     | 0.009881  |
| 27        | -15.8     | 0.01981   | 154.3     | 0.8764    | 0.9781    | 126.4     | 0.008656  |
| 28        | -16.85    | 0.05372   | 202.7     | 0.9624    | 0.9129    | 125.8     | 0.0007616 |
| 29        | -16.45    | 0.001535  | 204.6     | 0.8183    | 0.9172    | 120.0     | 0.001256  |
| 30        | -16.23    | 0.05626   | 191.1     | 0.8569    | 0.9398    | 121.4     | 0.00887   |
| 31        | -16.49

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 51        | -45.38    | 0.1       | 202.3     | 0.8915    | 0.9271    | 127.5     | 0.0001    |
| 52        | -16.84    | 0.05541   | 192.3     | 0.9032    | 0.9971    | 119.4     | 0.004392  |
| 53        | -16.16    | 0.04935   | 204.9     | 0.8834    | 0.9744    | 128.7     | 0.004229  |
| 54        | -271.7    | 0.005701  | 164.1     | 0.9864    | 0.9667    | 118.3     | 0.009098  |
| 55        | -17.36    | 0.07695   | 160.7     | 0.8407    | 0.9947    | 120.0     | 0.005941  |
| 56        | -15.42    | 0.05089   | 162.2     | 0.838     | 0.9218    | 121.6     | 0.001384  |
| 57        | -313.6    | 0.05953   | 167.9     | 0.9866    | 0.9396    | 116.7     | 0.007744  |
| 58        | -15.93    | 0.008178  | 189.8     | 0.9209    | 0.9891    | 129.8     | 0.009158  |
| 59        | -15.28    | 0.08648   | 160.8     | 0.8204    | 0.9315    | 120.0     | 0.00508   |
| 60        | -16.91    | 0.08698   | 206.7     | 0.8053    | 0.9616    | 119.5     | 0.004479  |
| 61        | -16.95

/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15000) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_p

| 74        | -50.92    | 1e-06     | 177.1     | 0.8       | 0.983     | 120.1     | 0.0001    |
| 75        | -16.16    | 0.04234   | 180.1     | 0.9336    | 0.9125    | 122.5     | 0.003437  |
| 76        | -17.68    | 1e-06     | 180.6     | 0.8785    | 1.0       | 123.9     | 0.01      |
| 77        | -15.63    | 0.08615   | 153.1     | 0.8422    | 0.9136    | 126.1     | 0.0009328 |
| 78        | -15.93    | 0.06506   | 179.7     | 0.8491    | 0.9657    | 125.2     | 0.00051   |
| 79        | -14.84    | 0.06043   | 189.7     | 0.9294    | 0.9209    | 121.2     | 0.002745  |
| 80        | -14.62    | 0.05843   | 158.6     | 0.9356    | 0.909     | 126.7     | 0.001283  |
| 81        | -46.17    | 0.01692   | 203.6     | 0.9773    | 0.9031    | 121.1     | 0.002129  |
| 82        | -20.85    | 0.03452   | 237.7     | 0.9875    | 0.9978    | 124.6     | 0.004294  |
| 83        | -17.39    | 0.01873   | 196.8     | 0.8       | 0.9       | 118.1     | 0.01      |
| 84        | -15.59

In [9]:
optimizer.max

{'target': -14.624831632476509,
 'params': {'alpha': 0.058428237542730146,
  'batch_size': 158.60572041092112,
  'beta_1': 0.9355582054882612,
  'beta_2': 0.9089543612935872,
  'hidden_size': 126.6933372337246,
  'learning_rate_init': 0.001283462649989831}}